### Passo 1 - Importar Arquivos e Bibliotecas

#### Importar Bibliotecas

In [ ]:
import pandas as pd
import win32com.client as win32
import pathlib

#### Importar Bases de Dados

In [ ]:
emails = pd.read_excel(r'Bases de Dados/Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados/Lojas.csv', encoding='latin', sep=';')
vendas = pd.read_excel(r'Bases de Dados/Vendas.xlsx')

#### Visualizar Bases e Informações

In [ ]:
display(emails)
display(lojas)
display(vendas)

In [ ]:
emails.info()
print( 50 * '-')
lojas.info()
print( 50 * '-')
vendas.info()

### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

### Tratamento de Dados

##### Incluir o nome da loja no DataFrame Vendas

In [ ]:
vendas = vendas.merge(lojas, on='ID Loja')
display(vendas)

#### Filtrar as Informações de Vendas por Loja

In [ ]:
   dicionario_lojas = {}

   for loja in lojas['Loja']:
    dicionario_lojas[loja] =  vendas.loc[vendas['Loja']== loja, :]

In [ ]:
display(dicionario_lojas['Iguatemi Esplanada'])

#### Definindo o Dia Indicador

In [ ]:
dia_indicador = vendas['Data'].max()

In [ ]:
print(dia_indicador)
print('{}/{}'.format(dia_indicador.day,dia_indicador.month))

### Passo 3 - Salvar a planilha na pasta de backup

#### Criando o Caminho para a Pasta Backup

In [ ]:
caminho_backup = pathlib.Path(r"Backup Arquivos Lojas")

#### Criar Pasta para Loja caso não Haja, e Salvar a Planilha da Loja na Pasta Correspondente

In [ ]:
arquivos_pasta_backup = caminho_backup.iterdir()
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()

    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.day,dia_indicador.month, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo
    dicionario_lojas[loja].to_excel(local_arquivo, index=False)

### Passo 4 - Calcular o Indicador e Automatizar para Todas as Lojas

#### Definição de Metas

In [ ]:
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedia_dia = 500
meta_ticketmedia_ano = 500


#### Executar Cálculos de cada Loja e Enviar por e-mail para o gerente Automatizando para todas as lojas

In [ ]:
for loja in dicionario_lojas:

    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']== dia_indicador, :]

    #faturamento ano
    faturamento_ano = vendas_loja['Valor Final'].sum()
    #faturamento dia
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    #diversidade de produtos
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())

    #ticket médio
    valor_venda = vendas_loja[['Código Venda','Quantidade','Valor Final']].groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    valor_venda_dia =  vendas_loja_dia[['Código Venda','Quantidade','Valor Final']].groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()

    #enviar e-mail
    outlook = win32.Dispatch('outlook.application')

    nome = emails.loc[emails['Loja']== loja, 'Gerente'].values[0]
    mail = outlook.CreateItem(0)
    mail.To = emails.loc[emails['Loja']== loja, 'E-mail'].values[0]
    mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'

    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = '#56fc03'
    else:
        cor_fat_dia = '#fc0320'

    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = '#56fc03'
    else:
        cor_fat_ano = '#fc0320'

    if qtde_produtos_dia >= meta_qtdeprodutos_dia:
        cor_qtde_dia = '#56fc03'
    else:
        cor_qtde_dia = '#fc0320'

    if qtde_produtos_ano >= meta_qtdeprodutos_ano:
        cor_qtde_ano = '#56fc03'
    else:
        cor_qtde_ano = '#fc0320'

    if ticket_medio_dia >= meta_ticketmedia_dia:
        cor_ticket_dia = '#56fc03'
    else:
        cor_ticket_dia = '#fc0320'

    if ticket_medio_ano >= meta_ticketmedia_ano:
        cor_ticket_ano = '#56fc03'
    else:
        cor_ticket_ano = '#fc0320'
    
    
    
    mail.HTMLBody = f'''
    
    <p>Bom dia {nome}, </p> 

    <p>Apresentado a seguir OnePage com o resultado da <strong>Loja {loja}</strong> no <strong>dia {dia_indicador.day}/{dia_indicador.month}</strong>:</p>

    <h2>ONEPAGE DIA</h2>
    <table font-family: poppis border="0.5" width="100%">
        <tr>
            <th>Indicador</th>
            <th>Meta</th>
            <th>Resultado</th>
            <th>Cenário Dia</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td style="text-align: center">R$ {meta_faturamento_dia:.2f}</td>
            <td style="text-align: center">R$ {faturamento_dia:.2f}</td>  
            <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
        </tr>    
        <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
            <td style="text-align: center">{qtde_produtos_dia}</td>
            <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">R$ {meta_ticketmedia_dia:.2f}</td> 
            <td style="text-align: center">R$ {ticket_medio_dia:.2f}</td>
            <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
        </tr>
    </table>
    <br>
    <h2>ONEPAGE ANO</h2>
    <table font-family: poppis, border="0.5" width="100%">
        <tr>
            <th>Indicador</th>
            <th>Meta</th>
            <th>Resultado</th>
            <th>Cenário Ano</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td style="text-align: right">R$ {meta_faturamento_ano:.2f}</td>
            <td style="text-align: right">R$ {faturamento_ano:.2f}</td>  
            <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
        </tr>    
        <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: right">{meta_qtdeprodutos_ano}</td>
            <td style="text-align: right">{qtde_produtos_ano}</td> 
            <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td style="text-align: right">R$ {meta_ticketmedia_ano:.2f}</td> 
            <td style="text-align: right">R$ {ticket_medio_ano:.2f}</td>
            <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
        </tr>
    </table>
    <br>

    <p>Em anexo encontra-se a planilha com todas as movimentações caso necessite de mais detalhes.</p>

    <p>Para quaisquers dúvidas fico á disposição.</p>
    <p>Cordialmente</p>
    <p>Rafael Oliveira</p>

    
    '''
    
    

    #anexos 
    attachment = pathlib.Path.cwd()/ caminho_backup / loja / f'{dia_indicador.day}_{dia_indicador.month}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))

    #enviar
    mail.Send()
    print( f'E-mail da loja {loja} enviado com sucesso.')


### Passo 5 - Criar ranking para diretoria

In [ ]:
faturamento_lojas = vendas[['Loja','Valor Final']].groupby('Loja').sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)

nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.day, dia_indicador.month)
faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

vendas_dia = vendas.loc[vendas['Data']== dia_indicador, :]
faturamento_lojas_dia = vendas_dia[['Loja','Valor Final']].groupby('Loja').sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)

nome_arquivo = '{}_{}_Ranking Dia.xlsx'.format(dia_indicador.day,dia_indicador.month)
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

### Passo 6 - Enviar e-mail para diretoria

In [ ]:
#enviar e-mail
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja']== 'Diretoria','E-mail'].values[0]
mail.Subject = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}'
mail.Body = f''' 
Prezados bom dia,

Apresentado abaixo o ranqueamento e resultados obtidos por Loja:

Melhor loja em faturamento do Dia: Loja {faturamento_lojas_dia.index[0]} com faturamento de R$ {faturamento_lojas_dia.iloc[0,0]:.2f}
Pior loja em faturamento do Dia: Loja {faturamento_lojas_dia.index[-1]} com faturamento de R$ {faturamento_lojas_dia.iloc[-1,0]:.2f}

Melhor loja em faturamento do Ano: Loja {faturamento_lojas_ano.index[0]} com faturamento de R$ {faturamento_lojas_ano.iloc[0,0]:.2f}
Pior loja em faturamento do Ano: Loja {faturamento_lojas_ano.index[-1]} com faturamento de R$ {faturamento_lojas_ano.iloc[-1,0]:.2f}

Para mais detalhes, consta em anexo os rankings completos do ano e do dia de todas as lojas.

Qualquer dúvida estou á disposição.

Cordialmente.
Rafael Oliveira

'''

#anexos
attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.day}_{dia_indicador.month}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))
attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.day}_{dia_indicador.month}_Ranking Dia.xlsx'
mail.Attachments.Add(str(attachment))

mail.Send()
print('E-mail da diretoria enviado com sucesso')
